In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.6 MB/s eta 0:00:00


In [9]:
! pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 6.3 MB/s eta 0:00:00:00:01


In [30]:
!pip install docx2txt

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=6ed42ef918892fd18ca44255e742550fc6f3e6f977b45a26d4dedc28845e48a7
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [39]:
import numpy as np
import pandas as pd
from docx import Document
import re

import os

import PyPDF2

In [5]:
def getMobileNumber(text) :
    patterns = [r'\b\d{10}\b',r'\b\d{5}-\d{5}\b',r'\b\d{5} \d{5}\b',
                r'\b\d{3}-\d{3}-\d{4}\b',r'\b\d{3} \d{3} \d{4}\b']
    for pattern in patterns :
        for line in text :
            match = re.search(pattern, line)
            if (match) :
                return match.group()
                break
    return None

def getName(text) :
    return None

def getEmailId(text) :
    patterns = [r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b']
    for pattern in patterns :
        for line in text :
            match = re.search(pattern, line)
            if (match) :
                return match.group()
                break
    return None

def getText(text) :
    newText = ' '.join(text)
    newText = newText.replace('\n','')
    return newText

In [6]:
def pdfParser(fileName) :
    
    pdfObj = open(fileName,'rb')
    
    pdfReader = PyPDF2.PdfReader(pdfObj)

    pageCount = len(pdfReader.pages)

    text = ""

    for i in range(pageCount):
        pageObj = pdfReader.pages[i]
        text += pageObj.extract_text()
        
    text = text.split('\n')
    
    newText = []

    for i in text :
        if len(i) > 3 :
            newText.append(i)
        
    return {'Name' : [getName(newText)],
                     'Mobile Number' : [getMobileNumber(newText)],
                     'Email Id' : [getEmailId(newText)],
                     'Resume Details' : [getText(newText)]}

In [7]:
def docxParser(filePath) :
    
    doc = Document(filePath)
    
    text = ""
    
    for para in doc.paragraphs:
        text += para.text
        
    text = text.split('\n')
    
    newText = []

    for i in text :
        if len(i) > 3 :
            newText.append(i)
        
    return {'Name' : [getName(newText)],
                     'Mobile Number' : [getMobileNumber(newText)],
                     'Email Id' : [getEmailId(newText)],
                     'Resume Details' : [getText(newText)]}

In [19]:
currDirectory = '/kaggle/input/resumes'

Resumes = os.path.join(currDirectory,os.listdir(currDirectory)[0])

dframe = pd.DataFrame(columns = ['Name','Mobile Number','Email Id','Resume Details'])

for file in os.listdir(Resumes) :
    fileType = file[-3:]
    filePath = os.path.join(Resumes,file)
    
    if fileType == 'pdf' :
        dicty = pdfParser(filePath)
        dicty['Name'] = file[:-4]
        frame = pd.DataFrame(dicty)
        dframe = pd.concat([dframe,frame])
        
    elif file[-4:] == 'docx' :
        dicty = docxParser(filePath)
        dicty['Name'] = file[:-5]
        frame = pd.DataFrame(dicty)
        dframe = pd.concat([dframe,frame])
        
    elif file[-3:] == 'doc' :
        
        
        dicty = docxParser(filePath)
        dicty['Name'] = file[:-5]
        frame = pd.DataFrame(dicty)
        dframe = pd.concat([dframe,frame])

In [21]:
dframe = dframe.reset_index().drop('index',axis = 1)

In [38]:
dframe.to_excel('output.xlsx', index=False)